In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterra as rt
import numpy as np
from pathlib import Path
import contextily as ctx
from matplotlib.colors import Normalize, PowerNorm

from measure_io import load_measure
from measure_io import create_measure

# global vars
CRS = 'EPSG:3310' # this is california albers

# file paths
RAW_DATA_DIR = Path("/Users/laurenwilner/Desktop/Desktop/epidemiology_PhD/01_data/raw/")

In [ ]:
# functions
def load_circuit(path):
    gdf = gpd.read_file(path)
    gdf = gdf.rename(columns = lambda c: c.lower()).dissolve("circuit_na")["geometry"].reset_index().to_crs(CRS)
    return gdf

def load_all_circuits():
    # clean polylines
    # delete extra cols
    # union polylines for a given circuit 
    # join all three provider circuit files into 1
    # NOTE: using polygon files since SDGE is only polygons. Polygons are Polylines with a 18m buffer.
    
    pge = load_circuit(RAW_DATA_DIR/ "psps_circuit_data/ICA_circuits/PGE/PGE_circuits_polygons/PGE_circuits_polygons.shp")
    sce = load_circuit(RAW_DATA_DIR/ "psps_circuit_data/ICA_circuits/SCE/SCE_circuits_polygon/SCE_circuits_polygon.shp")
    sdge = load_circuit(RAW_DATA_DIR/ "psps_circuit_data/ICA_circuits/SDGE_circuits/SDGE_circuits.shp")
    return pd.concat([pge,sce,sdge])

In [ ]:
# load and format data 

# ca census tracts
ca_ct = load_measure("ca_census_tracts").to_crs(CRS)
ca_shape = ca_ct.unary_union

# zcta boundaries
zcta_boundaries = load_measure("us_zcta_boundaries").to_crs(CRS)
print("loaded boundaries")
zcta_ca = zcta_boundaries[zcta_boundaries.intersects(ca_shape)]

# census tract boundaries
ct_ca = load_measure("ca_census_tracts").to_crs(CRS)

# psps data
psps = load_measure("us_circuit_psps_by_hr")
print("loaded psps")

# ca pop
ca_pop = rt.load_raster(RAW_DATA_DIR/ "CAPOP_2020_100m_TOTAL.tif").set_no_data_value(np.nan).to_crs(CRS)
print("loaded ca pop")

# load circuits
all_circuits = load_all_circuits()
print("loaded all circuits")

In [ ]:
ca_shape = ca_ct.unary_union
zcta_boundaries = load_measure("us_zcta_boundaries").to_crs(CRS)
zcta_ca = zcta_boundaries[zcta_boundaries.intersects(ca_shape)]
# create_measure(zcta_ca, "ca_zcta_shp")

In [ ]:
# ZCTA: DENOMINATOR (population)
# step 1: join zcta onto both pixel-level pop and all_circuits (denominator)
    # overlay pixels with zctas  (using sjoin because there are so many pixels per zcta)
    # overlay circuits with zctas  (using overlap since the circuit:zctas ratio is smaller)
ca_gdf = ca_pop.to_gdf()
ca_gdf = ca_gdf[ca_gdf["value"].notnull()]
ca_gdf = ca_gdf.reset_index().rename(columns = {"index":"pixel_id"})
ca_gdf = ca_gdf.sjoin(zcta_ca) # join zcta onto pop pixel 
circuits_zcta = all_circuits.overlay(zcta_ca, keep_geom_type = True) # join zctas  onto circuit df using an overlay

In [ ]:
# ZCTA DENOMINATOR (population)
# step 2: by zcta, overlay circuits with pop. then agg to zcta 
    # by zcta: 
        # turn raster data into a geodataframe
        # subset to only pixels with population in them
        # intersect this subset with the circuit polys 
        # proportionally split pixel pop if there is some of 2 circuits in a pixel
        # now we have the pop for each segment of each circuit so we can aggregate to circuit_name-zcta
        # end with df with zcta, geometry, pop, circuit_na

all_zcta_circuits = []
zctas = circuits_zcta["zcta"].unique()

for zcta in zctas:
    # filter df ZCTA
    circuits_z = circuits_zcta[circuits_zcta['zcta'] == zcta]
    ca_gdf_z = ca_gdf[ca_gdf['zcta'] == zcta]
    
    # overlay
    circuits = circuits_z.overlay(ca_gdf_z, keep_geom_type=True)

    # clean up
    # keep one zcta col
    circuits.rename(columns={'zcta_1': 'zcta'}, inplace=True)
    circuits.drop(columns=['zcta_2'], inplace=True, errors='ignore')
    # drop 'index_right'
    circuits.drop(columns=['index_right'], errors='ignore', inplace = True)
   
    # calculations
    circuits["intersect_area"] = circuits.area
    circuits["pixel_area"] = circuits.groupby("pixel_id")["intersect_area"].transform("sum")
    circuits["pop_weight"] = circuits["intersect_area"] / circuits["pixel_area"]
    circuits["intersect_population"] = circuits["value"] * circuits["pop_weight"]
    
    # sum up population for each circuit
    circuit_pop = circuits.groupby(["circuit_na", "zcta"])["intersect_population"].sum().reset_index()
    circuit_pop.rename(columns={'intersect_population': 'pop'}, inplace=True)
    
    # Append to list
    all_zcta_circuits.append(circuit_pop)

# concatenate all DataFrames into one
circuits_pop_zcta = pd.concat(all_zcta_circuits, ignore_index=True)

In [ ]:
# ZCTA DENOMINATOR
# write out clean geoparquet
# this probably wont be used in a synth control analysis but could be 
# used if we did indiv level exp-outcome analysis.
create_measure(circuits_pop_zcta, "ca_circuits_zcta_pop")

# write out geoparquet aggregated to zcta level to use for analysis for now
zcta_pop = circuits_pop_zcta.groupby("zcta")["pop"].agg("sum").reset_index()
create_measure(zcta_pop, "ca_gridded_zcta_pop")

In [ ]:
# CENSUS TRACT: DENOMINATOR (population)
# step 1: join zcta onto both pixel-level pop and all_circuits (denominator)
    # overlay pixels with zctas (using sjoin because there are so many pixels per zcta)
    # overlay circuits with zctas (using overlap since the circuit:zcta ratio is smaller)
ca_gdf = ca_pop.to_gdf()
ca_gdf = ca_gdf[ca_gdf["value"].notnull()]
ca_gdf = ca_gdf.reset_index().rename(columns = {"index":"pixel_id"})
ca_gdf = ca_gdf.sjoin(ct_ca) # join ct onto pop pixel 
circuits_ct = all_circuits.overlay(ct_ca, keep_geom_type = True) # join census tracts onto circuit df using an overlay

In [ ]:
# CENSUS TRACT DENOMINATOR (population)
# step 2: by census tract, overlay circuits with pop. then agg to ct 
    # by ct: 
        # turn raster data into a geodataframe
        # subset to only pixels with population in them
        # intersect this subset with the circuit polys 
        # proportionally split pixel pop if there is some of 2 circuits in a pixel
        # now we have the pop for each segment of each circuit so we can aggregate to circuit_name-ct
        # end with df with ct, geometry, pop, circuit_na

all_ct_circuits = []
cts = circuits_ct["geoid"].unique()

for ct in cts:
    # filter df ct
    circuits_c = circuits_ct[circuits_ct['geoid'] == ct]
    ca_gdf_c = ca_gdf[ca_gdf['geoid'] == ct]
    
    # overlay
    circuits = circuits_c.overlay(ca_gdf_c, keep_geom_type=True)

    # clean up
    # keep one ct col
    circuits.rename(columns={'geoid_1': 'geoid'}, inplace=True)
    circuits.drop(columns=['geoid_2'], inplace=True, errors='ignore')
    # drop 'index_right'
    circuits.drop(columns=['index_right'], errors='ignore', inplace = True)
   
    # calculations
    circuits["intersect_area"] = circuits.area
    circuits["pixel_area"] = circuits.groupby("pixel_id")["intersect_area"].transform("sum")
    circuits["pop_weight"] = circuits["intersect_area"] / circuits["pixel_area"]
    circuits["intersect_population"] = circuits["value"] * circuits["pop_weight"]
    
    # sum up population for each circuit
    circuit_pop = circuits.groupby(["circuit_na", "geoid"])["intersect_population"].sum().reset_index()
    circuit_pop.rename(columns={'intersect_population': 'pop'}, inplace=True)
    
    # Append to list
    all_ct_circuits.append(circuit_pop)

# concatenate all DataFrames into one
circuits_pop_ct = pd.concat(all_ct_circuits, ignore_index=True)

In [ ]:
# CENSUS TRACT DENOMINATOR
# write out clean geoparquet
# this probably wont be used in a synth control analysis but could be 
# used if we did indiv level exp-outcome analysis.
create_measure(circuits_pop_ct, "ca_circuits_ct_pop")

# write out geoparquet aggregated to ct level to use for analysis for now
ct_pop = circuits_pop_ct.groupby("geoid")["pop"].agg("sum").reset_index()
create_measure(ct_pop, "ca_gridded_ct_pop")

In [ ]:
# ZCTA NUMERATOR
# calculate the weight for each circuit-ZCTA overlap based on the proportion of the circuit that falls within the ZCTA.
circuits_zcta["intersect_area"] = circuits_zcta.area
circuits_zcta["circuit_area"] = circuits_zcta.groupby("circuit_na")["intersect_area"].transform("sum")
circuits_zcta["weight"] = circuits_zcta["intersect_area"] / circuits_zcta["circuit_area"]

# collapse psps to one row per event-circuit 
psps_collapse = psps[["circuit_name_ica", "psps_event_id", "total_customers_impacted"]].drop_duplicates()

# multiply the weight by the total_customers_impacted to find the customers_impacted for each ZCTA.
customers_impacted_zcta = psps_collapse.merge(circuits_zcta, how = 'left', left_on = 'circuit_name_ica', right_on = 'circuit_na')
customers_impacted_zcta["total_customers_impacted"] = customers_impacted_zcta["total_customers_impacted"] * customers_impacted_zcta["weight"]

# sum up the customers_impacted for circuits that fall within the same zcta
customers_impacted_zcta = customers_impacted_zcta.groupby(['zcta', 'psps_event_id'])['total_customers_impacted'].sum().reset_index()

# subset to rows with total_customers_impacted>0
customers_impacted_zcta = customers_impacted_zcta[customers_impacted_zcta["total_customers_impacted"]>0]

# merge back on psps metadata
# take the min outage start, max outage end, and recalc duration
psps_summary = pd.concat([
    psps.groupby('psps_event_id')['outage_start'].min(),
    psps.groupby('psps_event_id')['outage_end'].max(),
], axis=1)
psps_summary['duration'] = psps_summary['outage_end']-psps_summary['outage_start']
customers_impacted_zcta = customers_impacted_zcta.merge(psps_summary, how = 'inner', left_on = 'psps_event_id', right_on = 'psps_event_id')

customers_impacted_zcta

In [ ]:
# CENSUS TRACT NUMERATOR
# calculate the weight for each circuit-ZCTA overlap based on the proportion of the circuit that falls within the ZCTA.
circuits_ct["intersect_area"] = circuits_ct.area
circuits_ct["circuit_area"] = circuits_ct.groupby("circuit_na")["intersect_area"].transform("sum")
circuits_ct["weight"] = circuits_ct["intersect_area"] / circuits_ct["circuit_area"]

# collapse psps to one row per event
psps_collapse = psps[["circuit_name_ica", "psps_event_id", "total_customers_impacted"]].drop_duplicates()

# multiply the weight by the total_customers_impacted to find the customers_impacted for each ZCTA.
customers_impacted_ct = psps_collapse.merge(circuits_ct, how = 'left', left_on = 'circuit_name_ica', right_on = 'circuit_na')
customers_impacted_ct["total_customers_impacted"] = customers_impacted_ct["total_customers_impacted"] * customers_impacted_ct["weight"]

# sum up the customers_impacted for circuits that fall within the same zcta
customers_impacted_ct = customers_impacted_ct.groupby(['geoid', 'psps_event_id'])['total_customers_impacted'].sum().reset_index()

# subset to rows with total_customers_impacted>0
customers_impacted_ct = customers_impacted_ct[customers_impacted_ct["total_customers_impacted"]>0]

# merge back on psps metadata
# take the min outage start, max outage end, and recalc duration
psps_summary = pd.concat([
    psps.groupby('psps_event_id')['outage_start'].min(),
    psps.groupby('psps_event_id')['outage_end'].max(),
], axis=1)
psps_summary['duration'] = psps_summary['outage_end']-psps_summary['outage_start']
customers_impacted_ct = customers_impacted_ct.merge(psps_summary, how = 'inner', left_on = 'psps_event_id', right_on = 'psps_event_id')

customers_impacted_ct

In [ ]:
# write out parquet of numerator data
create_measure(customers_impacted_zcta, "ca_zcta_psps_customers_impacted")
create_measure(customers_impacted_ct, "ca_ct_psps_customers_impacted")

In [ ]:

# Notes
# 1. using polygons not polylines. these are polylines with 18m buffer. 
# 2. not all of our pop is covered by these circuits
    # ca pop covered by these circuits = 29399918
    # ca pop not covered by these circuits = 39538230
# 3. for any pixel with circuit coverage in this dataset: 
    # assuming that pixel is entirely covered by the circuits in the dataset
    # so, we are saying that on a pixel by pixel basis we do or do not have coverage
    # we can thus define the pop_weight as the relative size of each circuit segment in that pixel
# 4. there are circuits that cover no people
# 5. potential source of bias:
    # most zctas have 1000+ pixels covering them
    # ~300 have less than 1000 pixels in them
    # for now we will assume that most zctas have sufficient pixel coverage
